In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import sys
sys.path.insert(1, '..')
from cfb_api_key import api_key

In [2]:
csv_name = '../data/cfbd/games.csv'

In [61]:
class CfbDataManager(DataFrame):
    def __init__(self, df, *args, **kwargs):
        super().__init__(*args, **kwargs)
        for key in df:
            if "Unnamed" in key: continue
            self[key] = df[key]
        if 'id' in self: self.set_index('id',inplace=True)
    @classmethod
    def from_file(cls,file_name,*args, **kwargs):
        df = pd.read_csv(file_name)
        return cls(df,*args, **kwargs)
    def teams(self):
        return pd.concat([self['home_team'],self['away_team']]).unique()
    def view(self,**kwargs):
        df = self
        for key in kwargs:
            if key == "team": 
                df = df[(df['home_team']==kwargs[key]) | (df['away_team']==kwargs[key])]   
                continue
            assert key in df, "key {} not in dataframe".format(key)
            df = df[df[key]==kwargs[key]]
        return self.__class__(df)
    def occured_games(self):
        return self.__class__(self[(self['home_points'] > 0) & (self['away_points'] > 0)])
    def combine(self,df):
        if 'id' in df: df = df.set_index("id")
        assert df.index.name == "id", "index must be id"
        self = self[~self.index.isin(df.index)]
        df3 = pd.concat([self,df])
        return self.__class__(df3)
    def teams_and_scores(self):
        columns = ['away_team', 'home_team','away_points', 'home_points', 'neutral_site']
        return self.__class__(self[columns])
    def get_data(self,api_key,season,**kwargs):
        import cfbd
        from cfbd.rest import ApiException
        # Configure API key authorization: ApiKeyAuth
        configuration = cfbd.Configuration()
        configuration.api_key['Authorization'] = api_key
        configuration.api_key_prefix['Authorization'] = 'Bearer'
        # create an instance of the API class
        api_instance = cfbd.GamesApi(cfbd.ApiClient(configuration))
        games = api_instance.get_games(season, season_type='regular', **kwargs)
        games+=api_instance.get_games(season, season_type='postseason', **kwargs)
        games = [game.to_dict() for game in games]
        df = pd.DataFrame(games)
        return self.combine(df)

In [62]:
df = CfbDataManager.from_file('../data/cfbd/games.csv')

/home/ryan/Develop/Sports/cfb/env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (6,11) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [63]:
#df.get_data(api_key,2021).shape

In [71]:
df.view(season=2020).week.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16])

In [14]:
df_2.combine(df)

,season,week,season_type,start_date,start_time_tbd,neutral_site,conference_game,attendance,venue_id,venue,...,home_points,home_line_scores,home_post_win_prob,away_id,away_team,away_conference,away_points,away_line_scores,away_post_win_prob,excitement_index
id,,,,,,,,,,,,,,,,,,,,,
1,1869,1,regular,1869-11-06T00:00:00.000Z,NaN,False,NaN,NaN,NaN,NaN,...,6.0,[],NaN,163,Princeton,NaN,4.0,[],NaN,NaN
2,1869,2,regular,1869-11-13T00:00:00.000Z,NaN,False,NaN,NaN,NaN,NaN,...,8.0,[],NaN,164,Rutgers,NaN,0.0,[],NaN,NaN
3,1870,1,regular,1870-11-05T00:00:00.000Z,NaN,False,NaN,NaN,NaN,NaN,...,6.0,[],NaN,171,Columbia,NaN,3.0,[],NaN,NaN
4,1870,2,regular,1870-11-12T00:00:00.000Z,NaN,False,NaN,NaN,NaN,NaN,...,6.0,[],NaN,164,Rutgers,NaN,2.0,[],NaN,NaN
5,1872,1,regular,1872-11-02T00:00:00.000Z,NaN,False,NaN,NaN,NaN,NaN,...,0.0,[],NaN,171,Columbia,NaN,0.0,[],NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401310754,2021,13,regular,2021-11-27T05:00:00.000Z,True,False,True,NaN,5388.0,Canvas Stadium,...,NaN,NaN,NaN,2440,Nevada,Mountain West,NaN,NaN,NaN,NaN
401310755,2021,13,regular,2021-11-27T05:00:00.000Z,True,False,True,NaN,3971.0,University Stadium,...,NaN,NaN,NaN,328,Utah State,Mountain West,NaN,NaN,NaN,NaN
401310756,2021,13,regular,2021-11-27T05:00:00.000Z,True,False,True,NaN,3984.0,War Memorial Stadium,...,NaN,NaN,NaN,62,Hawai'i,Mountain West,NaN,NaN,NaN,NaN


In [15]:
df_2.shape

(79808, 23)

In [16]:
df.index.name

'id'

In [17]:
df

,season,week,season_type,start_date,start_time_tbd,neutral_site,conference_game,attendance,venue_id,venue,...,home_points,home_line_scores,home_post_win_prob,away_id,away_team,away_conference,away_points,away_line_scores,away_post_win_prob,excitement_index
id,,,,,,,,,,,,,,,,,,,,,
401238035,2020,1,regular,2020-09-04T00:00:00.000Z,False,False,False,NaN,3803,Legion Field,...,45.0,"[14, 14, 3, 14]",0.998081,2110,Central Arkansas,NaN,35.0,"[7, 14, 0, 14]",0.001919,2.226002
401207101,2020,1,regular,2020-09-04T01:00:00.000Z,False,False,False,NaN,3815,M. M. Roberts Stadium,...,21.0,"[0, 10, 3, 8]",0.269739,6,South Alabama,Sun Belt,32.0,"[13, 0, 10, 9]",0.730261,5.586950
401237353,2020,1,regular,2020-09-05T17:00:00.000Z,False,False,False,NaN,3775,Joan C. Edwards Stadium,...,59.0,"[21, 17, 14, 7]",0.998936,2198,Eastern Kentucky,NaN,0.0,"[0, 0, 0, 0]",0.001064,1.145603
401235700,2020,1,regular,2020-09-05T17:30:00.000Z,False,False,False,NaN,3841,Blaik Field at Michie Stadium,...,42.0,"[7, 14, 14, 7]",0.998940,2393,Middle Tennessee,Conference USA,0.0,"[0, 0, 0, 0]",0.001060,2.470913
401212484,2020,1,regular,2020-09-05T20:30:00.000Z,False,False,False,NaN,3644,Jim Wacker Field at Bobcat Stadium,...,24.0,"[0, 14, 7, 3]",0.046385,2567,SMU,American Athletic,31.0,"[0, 14, 14, 3]",0.953615,3.288221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401310754,2021,13,regular,2021-11-27T05:00:00.000Z,True,False,True,NaN,5388,Canvas Stadium,...,NaN,NaN,NaN,2440,Nevada,Mountain West,NaN,NaN,NaN,NaN
401310755,2021,13,regular,2021-11-27T05:00:00.000Z,True,False,True,NaN,3971,University Stadium,...,NaN,NaN,NaN,328,Utah State,Mountain West,NaN,NaN,NaN,NaN
401310756,2021,13,regular,2021-11-27T05:00:00.000Z,True,False,True,NaN,3984,War Memorial Stadium,...,NaN,NaN,NaN,62,Hawai'i,Mountain West,NaN,NaN,NaN,NaN


In [176]:
df_1 = pd.DataFrame({"a":[1,2],"b":[2,3]})
df_2 = pd.DataFrame({"a":[2,3],"b":[4,6]})
df_1.set_index('a',inplace=True)
df_2.set_index('a',inplace=True)
df_1

,b
a,
1,2
2,3


In [177]:
df_2

,b
a,
2,4
3,6


In [185]:
df_1.merge(df_2,how='outer',ignore_index=False)

TypeError: merge() got an unexpected keyword argument 'ignore_index'

In [196]:
df3 = pd.concat([df_1,df_2], join="outer")
df3 = df3[~df3.index.duplicated(keep='last')]
df3

,b
a,
1,2
2,4
3,6


In [ ]:
[~df3.index.duplicated(keep='first')]